In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Attention, Bidirectional, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [26]:
data = pd.read_csv("semeval2014.csv")

# Encode polarity labels (positive=2, neutral=1, negative=0)
label_encoder = LabelEncoder()
data['polarity'] = label_encoder.fit_transform(data['polarity'])

# Tokenize text and aspects
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Sentence'])

# Convert reviews and aspects to sequences
review_sequences = tokenizer.texts_to_sequences(data['Sentence'])
aspect_sequences = tokenizer.texts_to_sequences(data['Aspect Term'])

# Pad sequences
max_seq_len = 100  # Maximum length for reviews
review_padded = pad_sequences(review_sequences, maxlen=max_seq_len, padding='post', truncating='post')
aspect_padded = pad_sequences(aspect_sequences, maxlen=max_seq_len, padding='post', truncating='post')

# Extract labels
labels = tf.keras.utils.to_categorical(data['polarity'], num_classes=4)

# Split into training and testing datasets
train_reviews, test_reviews, train_aspects, test_aspects, train_labels, test_labels = train_test_split(
    review_padded, aspect_padded, labels, test_size=0.2, random_state=42
)


In [27]:
from tensorflow.keras.layers import Layer, Input, Embedding, Bidirectional, LSTM, Dense
import tensorflow as tf

# Define the AttentionLayer
class AttentionLayer(Layer):
    def __init__(self, lstm_units):
        super(AttentionLayer, self).__init__()
        self.lstm_units = lstm_units
        self.lstm = Bidirectional(LSTM(self.lstm_units, return_sequences=True))  # Create LSTM layer here

    def call(self, inputs):
        text_embedded, aspect_embedded = inputs
        
        # Process the text using a Bi-directional LSTM
        lstm_out = self.lstm(text_embedded)  # Apply LSTM to text embeddings
        
        # Aspect representation as query
        query = tf.reduce_mean(aspect_embedded, axis=1)  # shape: (batch_size, 128)
        
        # Ensure the query shape is compatible for matmul
        query = tf.expand_dims(query, axis=1)  # shape: (batch_size, 1, 128)
        
        # Attention mechanism
        attention_scores = tf.matmul(lstm_out, query, transpose_b=True)  # shape: (batch_size, seq_len, 1)
        attention_weights = tf.nn.softmax(attention_scores, axis=1)
        
        # Weighted sum of LSTM outputs based on attention weights
        context_vector = tf.reduce_sum(attention_weights * lstm_out, axis=1)  # shape: (batch_size, lstm_units * 2)
        
        return context_vector

# Model Architecture
def build_model(vocab_size, embedding_dim, max_seq_len, lstm_units):
    # Input layers
    review_input = Input(shape=(max_seq_len,))
    aspect_input = Input(shape=(max_seq_len,))
    
    # Embedding layer
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_seq_len)
    review_embedded = embedding_layer(review_input)
    aspect_embedded = embedding_layer(aspect_input)
    
    # Attention layer
    attention_layer = AttentionLayer(lstm_units)
    context_vector = attention_layer([review_embedded, aspect_embedded])
    
    # Fully connected layer
    dense_layer = Dense(64, activation='relu')(context_vector)
    output_layer = Dense(4, activation='softmax')(dense_layer)  # 4 classes for polarity

    # Model
    model = tf.keras.models.Model(inputs=[review_input, aspect_input], outputs=output_layer)
    return model

# Hyperparameters
vocab_size = 10000  # You can use the tokenizer's word index size
embedding_dim = 128
max_seq_len = 100
lstm_units = 64

# Build the model
model = build_model(vocab_size, embedding_dim, max_seq_len, lstm_units)

# Compile the model
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    [train_reviews, train_aspects], 
    train_labels, 
    epochs=5, 
    batch_size=32, 
    validation_data=([test_reviews, test_aspects], test_labels)
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate([test_reviews, test_aspects], test_labels)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


C:\Users\naman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5


C:\Users\naman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_45', 'keras_tensor_46']. Received: the structure of inputs=('*', '*')
  warnings.warn(


59/59 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.4227 - loss: 1.1852 - val_accuracy: 0.4025 - val_loss: 1.1169
Epoch 2/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.5563 - loss: 1.0343 - val_accuracy: 0.6208 - val_loss: 0.9015
Epoch 3/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 0.7361 - loss: 0.7042 - val_accuracy: 0.7055 - val_loss: 0.8174
Epoch 4/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.8343 - loss: 0.4779 - val_accuracy: 0.7034 - val_loss: 0.8445
Epoch 5/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.8678 - loss: 0.3783 - val_accuracy: 0.7013 - val_loss: 0.9107
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7391 - loss: 0.7982
Test Loss: 0.9106934070587158
Test Accuracy: 0.7012711763381958


## Make Predictions

In [28]:
# Sample test data (pre-tokenized and padded)
sample_reviews = [
    "The camera quality is amazing and the battery life is great.",  # Review 1
    "The screen resolution is poor, but the performance is decent.",  # Review 2
]

sample_aspects = [
    "camera",  # Aspect for Review 1
    "performance",  # Aspect for Review 2
]

# Convert these to padded sequences (you need to use the same tokenizer and padding as during training)
# Assuming `tokenizer` is already defined during your training process
max_seq_len = 100  # Use the same max length as used during training

sample_reviews_seq = tokenizer.texts_to_sequences(sample_reviews)
sample_reviews_padded = pad_sequences(sample_reviews_seq, maxlen=max_seq_len, padding='post', truncating='post')

sample_aspects_seq = tokenizer.texts_to_sequences(sample_aspects)
sample_aspects_padded = pad_sequences(sample_aspects_seq, maxlen=max_seq_len, padding='post', truncating='post')

# Convert to numpy arrays for model input
import numpy as np
sample_reviews_padded = np.array(sample_reviews_padded)
sample_aspects_padded = np.array(sample_aspects_padded)


In [29]:
# Make predictions on the sample data
predictions = model.predict([sample_reviews_padded, sample_aspects_padded])

# Convert predictions to readable class labels (assuming one-hot encoding with 4 classes)
predicted_classes = np.argmax(predictions, axis=1)

# Map the predicted classes back to the corresponding sentiment labels (e.g., 0: Positive, 1: Negative, 2: Neutral, 3: Mixed)
sentiment_labels = ['Positive', 'Negative', 'Neutral', 'Mixed']  # Adjust as needed based on your model's output
predicted_sentiments = [sentiment_labels[class_idx] for class_idx in predicted_classes]

# Display the results
for review, aspect, sentiment in zip(sample_reviews, sample_aspects, predicted_sentiments):
    print(f"Review: {review}")
    print(f"Aspect: {aspect}")
    print(f"Predicted Sentiment: {sentiment}")
    print("-" * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
Review: The camera quality is amazing and the battery life is great.
Aspect: camera
Predicted Sentiment: Mixed
--------------------------------------------------
Review: The screen resolution is poor, but the performance is decent.
Aspect: performance
Predicted Sentiment: Negative
--------------------------------------------------
